# Week 13 - Sequential Decision Making 2 (reinforcement learning)
## 1. Monte Carlo prediction

In this notebook, you will be tested on your capacity to **complete the Monte Carlo prediction algortihm**. <br>
You can find details about the algorithm at slide 13 of the slide deck. <br>
The algorithm will be tested on Blackjack as seen in page 15 <br>

The code was Adapted from: https://github.com/dennybritz/reinforcement-learning/tree/master/MC

### 1.1 Setup

In [1]:
# imports
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys

from collections import defaultdict

from blackjack import BlackjackEnv
import plotting

matplotlib.style.use('ggplot')

In [2]:
env = BlackjackEnv()

### 1.2 Monte Carlo prediction  - code completion

Now comes the code completion part. <br>
You need to write a function that evaluates the values of each states given a policy. <br>
Refer to the algorithm in slide 13 to complete the function below. <br>


In [4]:
def mc_prediction(policy, env, num_episodes, discount_factor=1.0):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given policy using sampling.
    
    Args:
        policy: A function that maps an observation to action probabilities.
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # Keeps track of sum and count of returns for each state
    # to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    # The final value function
    V = defaultdict(float)
    
    # Implement this!

    return V    

Now, we will define a simple policy which we will evaluate. <br>
This policy is the same as the one presented is slide 16. <br>
Speciically, the policy hits except when the sum of the card is 20 or 21.

In [5]:
def sample_policy(observation):
    """
    A policy that sticks if the player score is > 20 and hits otherwise.
    """
    score, dealer_score, usable_ace = observation
    return 0 if score >= 20 else 1

We now evaluate the policy for 10k and 500k iterations. <br>
The resulting figures should look like the ones in slide 16.

In [7]:
V_10k = mc_prediction(sample_policy, env, num_episodes=10000)
plotting.plot_value_function(V_10k, title="10,000 Steps")

V_500k = mc_prediction(sample_policy, env, num_episodes=500000)
plotting.plot_value_function(V_500k, title="500,000 Steps")

ValueError: min() arg is an empty sequence